In [1]:
import pandas as pd, numpy as np

In [2]:
import os

os.getcwd()

'C:\\Users\\Debjata'

In [3]:
df = pd.DataFrame(index = np.arange(1_00_000), columns = np.arange(1))

In [4]:
df.insert(0, 'ID', range(0 + len(df)))

In [5]:
### making sourcing month variable

num_rows = 1_00_000

percent_s1 = 0.1
percent_s2 = 0.15
percent_s3 = 0.2
percent_s4 = 0.2
percent_s5 = 0.2
percent_s6 = 0.15


num_1 = int(num_rows * percent_s1)
num_2 = int(num_rows * percent_s2)
num_3 = int(num_rows * percent_s3)
num_4 = int(num_rows * percent_s4)
num_5 = int(num_rows * percent_s5)
num_6 = int(num_rows * percent_s6)

column_data = ['2024-01-01'] * num_1 + ['2024-02-01'] * num_2 + ['2024-03-01'] * num_3 + ['2024-04-01'] * num_4 + ['2024-05-01'] * num_5 + ['2024-06-01'] * num_6


np.random.shuffle(column_data)
df1 = pd.DataFrame(column_data)
df1.rename(columns = {0:'sourcing_month'},inplace=True)

In [6]:
df = df[['ID']]
df = pd.concat([df,df1], axis=1)

In [7]:
df['sourcing_month'].value_counts(dropna=False)

sourcing_month
2024-03-01    20000
2024-05-01    20000
2024-04-01    20000
2024-02-01    15000
2024-06-01    15000
2024-01-01    10000
Name: count, dtype: int64

In [8]:
### making geo month variable

num_rows = 1_00_000

percent_s1 = 0.2
percent_s2 = 0.3
percent_s3 = 0.4
percent_s4 = 0.1


num_1 = int(num_rows * percent_s1)
num_2 = int(num_rows * percent_s2)
num_3 = int(num_rows * percent_s3)
num_4 = int(num_rows * percent_s4)


state_data = ['WB'] * num_1 + ['MH'] * num_2 + ['TN'] * num_3 + ['UP'] * num_4


np.random.shuffle(state_data)
df1 = pd.DataFrame(state_data)
df1.rename(columns = {0:'state'},inplace=True)

df = pd.concat([df,df1], axis=1)

In [9]:
df['state'].value_counts(dropna=False)

state
TN    40000
MH    30000
WB    20000
UP    10000
Name: count, dtype: int64

In [10]:
### making channel month variable

num_rows = 1_00_000

percent_s1 = 0.2
percent_s2 = 0.6
percent_s3 = 0.1
percent_s4 = 0.1


num_1 = int(num_rows * percent_s1)
num_2 = int(num_rows * percent_s2)
num_3 = int(num_rows * percent_s3)
num_4 = int(num_rows * percent_s4)


channel_data = ['ch1'] * num_1 + ['ch2'] * num_2 + ['ch3'] * num_3 + ['ch4'] * num_4


np.random.shuffle(channel_data)
df1 = pd.DataFrame(channel_data)
df1.rename(columns = {0:'channel'},inplace=True)

df = pd.concat([df,df1], axis=1)

In [11]:
### making segment month variable

num_rows = 1_00_000

percent_s1 = 0.2
percent_s2 = 0.6
percent_s3 = 0.2



num_1 = int(num_rows * percent_s1)
num_2 = int(num_rows * percent_s2)
num_3 = int(num_rows * percent_s3)



income_data = ['high'] * num_1 + ['med'] * num_2 + ['low'] * num_3 


np.random.shuffle(income_data)
df1 = pd.DataFrame(income_data)
df1.rename(columns = {0:'income_seg'},inplace=True)

df = pd.concat([df,df1], axis=1)

In [12]:
# Define base weights
state_weight = {'MH': 1.2, 'TN': 1.0, 'UP': 0.9, 'WB': 0.85}
channel_weight = {'ch1': 1.2, 'ch2': 1.0, 'ch3': 0.8, 'ch4': 0.6}
income_weight = {'high': 1.3, 'med': 1.0, 'low': 0.7}

# Assign ticket size
np.random.seed(42)  # For reproducibility
base_amount = 10000  # You can adjust this

df['ticket_size'] = df.apply(
    lambda row: round(
        base_amount *
        state_weight.get(row['state'], 0.8) *
        channel_weight.get(row['channel'], 1.0) *
        income_weight.get(row['income_seg'], 1.0) *
        np.random.normal(1, 0.05), 2
    ),
    axis=1
)
df['ticket_size'] = df['ticket_size'].round(0)


In [13]:
state_age = {'MH': 1.1, 'TN': 1.0, 'UP': 0.95, 'WB': 0.9}
channel_age = {'ch1': 1.1, 'ch2': 1.0, 'ch3': 0.9, 'ch4': 0.7}
income_age = {'high': 1.2, 'med': 1.0, 'low': 0.8}

# Base age and noise
base_age = 30
np.random.seed(42)

df['age'] = df.apply(
    lambda row: int(
        base_age *
        state_age.get(row['state'], 0.9) *
        channel_age.get(row['channel'], 1.0) *
        income_age.get(row['income_seg'], 1.0) *
        np.random.normal(1, 0.05)
    ),
    axis=1
)

# Optional: cap age between 21 and 65
df['age'] = df['age'].clip(lower=21, upper=65)


In [14]:
#make default flag

# Define base risk weights
state_risk = {'WB': 1.3, 'UP': 1.1, 'TN': 1.0, 'MH': 0.9}
channel_risk = {'ch1': 2, 'ch2': 1.0, 'ch3': 0.6, 'ch4': 0.2}
income_risk = {'low': 2.5, 'med': 1.0, 'high': 0.2}

# Base probability of default
base_prob = 0.05  # 5% base rate
np.random.seed(42)

# Calculate probability and assign flag
def assign_dpd(row):
    prob = base_prob * \
           state_risk.get(row['state'], 1.0) * \
           channel_risk.get(row['channel'], 1.0) * \
           income_risk.get(row['income_seg'], 1.0)
    return int(np.random.rand() < prob)

df['dpd_90_ever'] = df.apply(assign_dpd, axis=1)


In [15]:
df['dpd_90_ever'].value_counts()

dpd_90_ever
0    93581
1     6419
Name: count, dtype: int64

In [16]:
df.groupby('state')['dpd_90_ever'].mean()

state
MH    0.054233
TN    0.063675
UP    0.064700
WB    0.079900
Name: dpd_90_ever, dtype: float64

In [17]:
df.groupby('channel')['dpd_90_ever'].mean()

channel
ch1    0.118300
ch2    0.059367
ch3    0.036000
ch4    0.013100
Name: dpd_90_ever, dtype: float64

In [18]:
df.groupby('income_seg')['dpd_90_ever'].mean()

income_seg
high    0.011500
low     0.142600
med     0.055617
Name: dpd_90_ever, dtype: float64

In [19]:
np.random.seed(42)

### 8. Branch Visit Frequency (0–10)
def assign_branch_visits(channel):
    if channel == 'ch1':
        return np.random.poisson(3.5)  # Highest
    elif channel == 'ch2':
        return np.random.poisson(2.0)
    elif channel == 'ch3':
        return np.random.poisson(1.0)
    else:  # ch4
        return np.random.binomial(1, 0.3)  # Mostly low

df['branch_visit_freq'] = df['channel'].apply(assign_branch_visits)
df['branch_visit_freq'] = df['branch_visit_freq'].clip(0, 10)

### 9. Referral Source
def assign_referral(row):
    if row['channel'] == 'ch1':
        return np.random.choice(['Branch', 'Agent'], p=[0.6, 0.4])
    elif row['channel'] == 'ch3':
        return np.random.choice(['Online Ad', 'Friend'], p=[0.7, 0.3])
    elif row['channel'] == 'ch2':
        return np.random.choice(['Branch', 'Agent', 'Online Ad', 'Friend'], p=[0.3, 0.3, 0.2, 0.2])
    else:  # ch4
        return np.random.choice(['Online Ad', 'Friend', 'Agent'], p=[0.5, 0.3, 0.2])

df['referral_source'] = df.apply(assign_referral, axis=1)

### 10. Document Completeness Flag
def assign_doc_flag(row):
    prob = 0.6  # base probability
    if row['channel'] == 'ch1':
        prob += 0.15
    if row['state'] == 'MH':
        prob += 0.1
    if row['income_seg'] == 'high':
        prob += 0.15
    return int(np.random.rand() < min(prob, 0.95))  # cap at 95%

df['doc_complete_flag'] = df.apply(assign_doc_flag, axis=1)

# Preview
# print(df[['ID', 'channel', 'state', 'income_seg', 'branch_visit_freq', 'referral_source', 'doc_complete_flag']].head())

In [20]:
df.columns.tolist()

['ID',
 'sourcing_month',
 'state',
 'channel',
 'income_seg',
 'ticket_size',
 'age',
 'dpd_90_ever',
 'branch_visit_freq',
 'referral_source',
 'doc_complete_flag']

In [21]:
df = df[['ID',
 'sourcing_month',
 'state',
 'channel',
 'income_seg',
 'ticket_size',
 'age',
 'branch_visit_freq',
 'referral_source',
 'doc_complete_flag', 'dpd_90_ever',]]

In [22]:
df['doc_complete_flag']= df['doc_complete_flag'].astype('str')+ '_'

In [23]:
df.to_csv('dummy_credit_risk_data.csv', index=False)

In [24]:
df.to_excel('dummy_credit_risk_data.xlsx', index=False)